In [161]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re

# 서울 구 지도

<img src = '서울구지도.png'>

In [ ]:
'''
강서구 : 우장산역(스마트), 까치산역(스마트), 마곡역(스마트), 강서구청 본관 - 스마트 도서관
양천구 : 목동역(U-도서관), 양천도서관 현관(스마트), 오목교(스마트 도서관)
구로구 : 천왕역(스마트), 신도림(스마트), 개봉역(스마트)
영등포구 : 여의도역, 제1/2 스포츠센터 => 스마트도서관
동작구 : 신대방삼거리역, 장승배기역, 총신대입구역, 노들역 => 스마트
금천구 : 금천고가(스마트)
관악구 : 신림역(스마트), 서울대입구, 신대방, 봉천, 낙성대, 관악중앙도서관=> U-도서관
서초구 : 내방역(스마트), 양재역(스마트), 
강남구 : 강남구청역, 논현1동, 삼성2동, 청담역
송파구 : 잠실나루역
강동구 : x
마포구 : x
서대문구 : 아현역, 홍제역, 독립문역
은평구 : 연신내역, 불광역, 역촌역
종로구 : x
중구 : 충정로역, 중립동 주민센터
용산구 : 남산도서관
성북구 : 석계역, 고려대역 / U-도서관이 활발(12개)
강북구 : x / U-도서관
성동구 : 금호역, 옥수역, 상왕십리
동대문구 : 한국외대 (스마트)
도봉구 : 쌍문역(U-도서관)
노원구 : 노원평생학습관(스마트)
중랑구 : 용마산역, 망우역, 중화역(스마트)
광진구 : 광진정보도서관(스마트)
'''

# 데이터셋 구축

In [2]:
smart = pd.DataFrame()

In [111]:
smart['gu'] = ['강서구']*4 + ['양천구']*3 + ['구로구']*3 + ['영등포구']*3 + ['동작구']*4 + ['금천구']\
                + ['관악구']*6 + ['서초구']*2 + ['강남구']*4 + ['송파구'] + ['강동구'] + ['마포구']\
                + ['서대문구']*3 + ['은평구']*3 + ['종로구'] + ['중구']*2 + ['용산구'] + ['성북구']*2\
                + ['강북구'] + ['성동구']*3 + ['동대문구'] + ['도봉구'] + ['노원구'] + ['중랑구']*3 + ['광진구']
smart['location'] = ['우장산역','까치산역','마곡역','강서구청 본관','목동역','양천도서관','오목교역'
                     ,'천왕역','신도림역','개봉역','여의도역','제1스포츠센터','제2스포츠센터'
                    ,'신대방삼거리역','장승배기역','총신대입구역','노들역','금천고가','신림역','서울대입구역','봉천역'
                    ,'신대방역','낙성대역','관악중앙도서관','내방역','양재역','강남구청역','논현1동','삼성2동','청담역'
                    ,'잠실나루역',np.nan,np.nan,'아현역','홍제역','독립문역','연신내역','불광역','역촌역'
                    ,np.nan,'충정로역','중립동_주민센터','남산도서관','석계역','고려대역',np.nan,'금호역'
                    ,'옥수역','상왕십리역','한국외대','쌍문역','노원평생학습관','용마산역','망우역','중화역'
                    ,'광진정보도서관']

In [19]:
smart['whether'] = [2]*19 + [1]*5 + [2]*7 +[np.nan]*2 + [2]*6 + [np.nan] + [2]*5 + [np.nan]\
                    +[2]*10

In [ ]:
smart['location2'] = smart['gu'] + ' ' +smart['location']

# 위치 좌표 끌어오기

    - 네이버 셀레늄 -> 주소 수집 -> 카카오 api

In [147]:
smart.head()

,gu,location,whether,location2,addr
0,강서구,우장산역,2.0,강서구 우장산역,서울 강서구 강서로 262
1,강서구,까치산역,2.0,강서구 까치산역,서울 강서구 강서로 54
2,강서구,마곡역,2.0,강서구 마곡역,서울 강서구 공항대로 163
3,강서구,강서구청 본관,2.0,강서구 강서구청 본관,error
4,양천구,목동역,2.0,양천구 목동역,서울 양천구 오목로 245


In [148]:
base_url = 'https://map.naver.com/'

browser = webdriver.Chrome('/Users/sinmin-yong/GitHub/공모전/도서빅데이터/chromedriver')
browser.get(base_url)
time.sleep(1.5)

In [157]:
addr_ls = []
for idx, loc in enumerate(smart['location2'].tolist()):
    
    if str(loc) == 'nan':
        addr_ls.append(np.nan)
        pass
    
    else:
        try:
            
            '''browser.switch_to_default_content()

            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].clear()

            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].send_keys(loc)
            time.sleep(0.2)
            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].send_keys(Keys.ENTER)
            time.sleep(1.5)

            browser.switch_to_frame('searchIframe')

            addr = browser.find_elements_by_xpath('//*[@id = "app-root"]//ul/li[1]')[0].text.split('\n')[1]

            addr_ls.append(addr)'''
            
            browser.switch_to_default_content()

            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].clear()

            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].send_keys(loc)
            time.sleep(0.2)
            browser.find_elements_by_xpath('//search-input-box/div/div/div/input')[0].send_keys(Keys.ENTER)
            time.sleep(1.5)

            browser.switch_to_frame('searchIframe')
            
            try:
                browser.find_elements_by_xpath('//*[@id = "app-root"]//ul/li[1]/div[2]/div[2]/span')[0].click()
                time.sleep(0.8)

                addr = browser.find_elements_by_xpath('//*[@id = "app-root"]//ul/li[1]/div[2]/div[2]/div/div[1]')[0].text

                addr_ls.append(addr)
            except:
                browser.find_elements_by_xpath('//*[@id = "app-root"]//ul/li[1]/div[1]/div[2]/span')[0].click()
                time.sleep(0.8)

                addr = browser.find_elements_by_xpath('//*[@id = "app-root"]//ul/li[1]/div[1]/div[2]/div/div[1]')[0].text

                addr_ls.append(addr)
            
        except:
            addr_ls.append('error')
        
    time.sleep(1)

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: use driver.switch_to.default_content instead
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: use driver.switch_to.frame instead


In [159]:
smart['addr2'] = addr_ls

In [170]:
smart.fillna('error',inplace = True)

In [173]:
smart['addr3'] = smart['addr2'].apply(lambda x : re.sub('도로명|복사','',x))

In [214]:
smart

,gu,location,whether,location2,addr,addr2,addr3,x,y
0,강서구,우장산역,2,강서구 우장산역,서울 강서구 강서로 262,도로명서울 강서구 강서로 262복사,서울 강서구 강서로 262,126.836408931785,37.5490001953873
1,강서구,까치산역,2,강서구 까치산역,서울 강서구 강서로 54,도로명서울 강서구 강서로 54복사,서울 강서구 강서로 54,126.846611592059,37.5319560847746
2,강서구,마곡역,2,강서구 마곡역,서울 강서구 공항대로 163,도로명서울 강서구 공항대로 163복사,서울 강서구 공항대로 163,126.825547992125,37.5601984535897
3,강서구,강서구청 본관,2,강서구 강서구청 본관,error,error,error,error,error
4,양천구,목동역,2,양천구 목동역,서울 양천구 오목로 245,도로명서울 양천구 오목로 245복사,서울 양천구 오목로 245,126.864742584692,37.52619868083
5,양천구,양천도서관,2,양천구 양천도서관,3,도로명서울 양천구 목동서로 113복사,서울 양천구 목동서로 113,126.876230784641,37.5335602034508
6,양천구,오목교역,2,양천구 오목교역,서울 양천구 오목로 342,도로명서울 양천구 오목로 342복사,서울 양천구 오목로 342,126.875354626254,37.5244500703802
7,구로구,천왕역,2,구로구 천왕역,서울 구로구 천왕로1길 60,도로명서울 구로구 천왕로1길 60복사,서울 구로구 천왕로1길 60,126.834202124109,37.4826379965713
8,구로구,신도림역,2,구로구 신도림역,8,도로명서울 구로구 경인로 662복사,서울 구로구 경인로 662,126.888212953215,37.5081230677129
9,구로구,개봉역,2,구로구 개봉역,서울 구로구 경인로40길 47,도로명서울 구로구 경인로40길 47복사,서울 구로구 경인로40길 47,126.859102583368,37.4945222528654


In [189]:
result_ls = []
for idx,loc in enumerate(smart['addr3'].tolist()):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+loc
    headers = {"Authorization": "KakaoAK 74e28dfd4b8f3d68ffb593d320d338d1"}
    result = json.loads(str(requests.get(url,headers=headers).text))
    ls = [result]
    
    result_ls.append(ls)

In [209]:
y_ls = []
x_ls = []

for i in range(len(result_ls)):
    
    try:
        x = result_ls[i][0]['documents'][0]['road_address']['x']
        y = result_ls[i][0]['documents'][0]['road_address']['y']
        
        x_ls.append(x)
        y_ls.append(y)
    except:
        x_ls.append('error')
        y_ls.append('error')

In [211]:
smart['x'] = x_ls
smart['y'] = y_ls

In [212]:
smart

,gu,location,whether,location2,addr,addr2,addr3,x,y
0,강서구,우장산역,2,강서구 우장산역,서울 강서구 강서로 262,도로명서울 강서구 강서로 262복사,서울 강서구 강서로 262,126.836408931785,37.5490001953873
1,강서구,까치산역,2,강서구 까치산역,서울 강서구 강서로 54,도로명서울 강서구 강서로 54복사,서울 강서구 강서로 54,126.846611592059,37.5319560847746
2,강서구,마곡역,2,강서구 마곡역,서울 강서구 공항대로 163,도로명서울 강서구 공항대로 163복사,서울 강서구 공항대로 163,126.825547992125,37.5601984535897
3,강서구,강서구청 본관,2,강서구 강서구청 본관,error,error,error,error,error
4,양천구,목동역,2,양천구 목동역,서울 양천구 오목로 245,도로명서울 양천구 오목로 245복사,서울 양천구 오목로 245,126.864742584692,37.52619868083
5,양천구,양천도서관,2,양천구 양천도서관,3,도로명서울 양천구 목동서로 113복사,서울 양천구 목동서로 113,126.876230784641,37.5335602034508
6,양천구,오목교역,2,양천구 오목교역,서울 양천구 오목로 342,도로명서울 양천구 오목로 342복사,서울 양천구 오목로 342,126.875354626254,37.5244500703802
7,구로구,천왕역,2,구로구 천왕역,서울 구로구 천왕로1길 60,도로명서울 구로구 천왕로1길 60복사,서울 구로구 천왕로1길 60,126.834202124109,37.4826379965713
8,구로구,신도림역,2,구로구 신도림역,8,도로명서울 구로구 경인로 662복사,서울 구로구 경인로 662,126.888212953215,37.5081230677129
9,구로구,개봉역,2,구로구 개봉역,서울 구로구 경인로40길 47,도로명서울 구로구 경인로40길 47복사,서울 구로구 경인로40길 47,126.859102583368,37.4945222528654


In [213]:
smart.to_csv('스마트,U도서관데이터.csv',index = False)